In [1]:
# Подключаем необходимые для работы библиотеки
import numpy as np
import pandas as pd
import os
import gdown
import zipfile
import warnings
from google.colab import drive

In [2]:
# Отключаем предупреждения
warnings.filterwarnings('ignore')

In [3]:
# Монтируем диск для скачивания, распаковки и работы с CSV-файлом
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Скачиваем ZIP-файл с записями о контрактах с Google-диска
url = 'https://drive.google.com/uc?id=1sRHx27O3NgTivrrQHdBdTAqxdCYNmARW'
name_zip = 'fz.zip'
gdown.download(url, name_zip, quiet=False)

In [9]:
# Распаковываем ZIP-файл с записями о контрактах
with zipfile.ZipFile(name_zip, 'r') as zip_ref:
  zip_ref.extractall()

In [4]:
# Читаем данные из супербольшого CSV-файла в датафрейм кусками, одновременно удаляя ненужные для анализа столбцы и строки с пустыми значениями
df_original = pd.DataFrame()
name_csv = 'fz.csv'
type_cols = {'Subject': 'string', 'OKPD2': 'string'}
with pd.read_csv(name_csv, chunksize=100000, encoding='utf-8', on_bad_lines="skip", delimiter=',', header=None, usecols=[11, 26], names=['Subject', 'OKPD2'], dtype=type_cols) as reader:
  for chunk in reader:
    # Отфильтровываем записи о контрактах, оставляя записи групп ОКПД 41, 42, 43, 71.1
    chunk = chunk[chunk['OKPD2'].str.contains('^41.') | chunk['OKPD2'].str.contains('^42.') | chunk['OKPD2'].str.contains('^43.') | chunk['OKPD2'].str.contains('^71.1')]
    # Удаляем из датафрейма строки с неопределенным предметом закупки и строки с пустыми значениями
    chunk = chunk.loc[chunk['Subject'] != '<НЕ ОПРЕДЕЛЕНО>']
    chunk.dropna
    df_original = pd.concat([df_original, chunk], ignore_index=True)

In [5]:
df_work = df_original

In [6]:
# Сбрасываем индексы в датафрейме
df_work = df_work.reset_index()
del df_work['index']

In [11]:
df_work.sample(10)

,Subject,OKPD2
723176,Оказание услуг по техническому обслуживанию ли...,43.2
1232417,"Выполнение работ, в результате которых обеспеч...",71.1
1192389,«Капитальный ремонт систем теплоснабжения МО «...,43.2
1302262,Оказание услуг по техническому обслуживанию и ...,43.2
951491,Выполнение работ по отсыпке автомобильных доро...,42.1
212679,Приобретение у застройщика жилого помещения (д...,41.2
724436,"Установка оконных решеток, оконных фрамуг в ин...",43.3
1415566,Ремонт кровли лит. А (основн . строение) здани...,43.9
537744,"Устройство систем видеонаблюдения, аудиооповещ...",43.2
287425,Оказание услуг по разработке эскизного проекта...,71.1


In [8]:
df_work.shape

(1764647, 2)

In [9]:
df_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764647 entries, 0 to 1764646
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Subject  string
 1   OKPD2    string
dtypes: string(2)
memory usage: 26.9 MB
